In [ ]:
!pip install tiktoken

In [ ]:
import os
import numpy as np
import pandas as pd
os.chdir('/kaggle/input/extlibs/.venv/Lib/site-packages')
import pydub
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs
import torch
os.chdir('/kaggle/input')
from transcribe import transcribe
import transcribe.transcribe as transcriber
from tokenizer import Tokenizer, get_tokenizer
from model import (Whispersa,TextDecoder,AudioEncoder,ModelDimensions)

In [ ]:
tokenizer =  get_tokenizer()
# Create an instance of ModelDimensions
dimensions_instance = ModelDimensions(
       n_mels=80, n_audio_ctx=1500, n_audio_state=768, n_audio_head=12, n_audio_layer=12, n_vocab=tokenizer.encoding.n_vocab, n_text_ctx=448, n_text_state=768, n_text_head=12, n_text_layer=12

)
device = "cpu" #"cpu"
model = Whispersa(dimensions_instance).to(device)
save_path = "/kaggle/input/speech-pro1/model3.pt"  # Replace with your desired path
model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))
model.to(device)

In [ ]:
file_name = os.listdir("/kaggle/input/testdata")[0]
result = transcriber.transcribe(model, fr'/kaggle/input/testdata/{file_name}', language='en', temperature=1.0, word_timestamps=True);

In [ ]:
f = open("/kaggle/working/transcription.txt", "w+")
f.write(result['text'])
f.close()

#open and read the file after the appending:
f = open("/kaggle/working/transcription.txt", "r")
print(f.read())

In [ ]:
def text_to_speech_file(text: str) -> str:
    # Calling the text_to_speech conversion API with detailed parameters
    client = ElevenLabs(api_key="sk_00ec341f60b35841ef2468cada45b6e20003f3436beecb00")
    response = client.text_to_speech.convert(
        voice_id="pNInz6obpgDQGcFmaJgB", # Adam pre-made voice
        optimize_streaming_latency="0",
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_turbo_v2", # use the turbo model for low latency, for other languages use the `eleven_multilingual_v2`
        voice_settings=VoiceSettings(
            stability=0.0,
            similarity_boost=1.0,
            style=0.0,
            use_speaker_boost=True,
        ),
    )
    # Generating a unique file name for the output MP3 file
    save_file_path = "/kaggle/working/speech.mp3"
    # Writing the audio to a file
    with open(save_file_path, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

In [ ]:
result['text'] = result['text'].replace('stutter', '')
text_to_speech_file(result['text'])